# Hyperparameter Tuning

In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
# importlib.reload(src.model)

# Example for loading data (you can skip this if data is already loaded)
train_data_path = '../data/CMaps/train_FD001.txt'
test_data_path = '../data/CMaps/test_FD001.txt'
rul_data_path = '../data/CMaps/RUL_FD001.txt'

# Create column names
col_names = [
    'engine_id', 'time_in_cycles', 
    'operational_setting_1', 'operational_setting_2', 'operational_setting_3'
] + [f'sensor_{i}' for i in range(1, 27)]  # This creates sensor_1 to sensor_21

# Read the data into pandas dataframes
train_df = pd.read_csv(train_data_path, sep=' ', header=None, names=col_names)
test_df = pd.read_csv(test_data_path, sep=' ', header=None, names=col_names)
rul_df = pd.read_csv(rul_data_path, header=None, names=['RUL'])

In [3]:
# Drop NaN values
train_df.dropna(axis=1, how='all', inplace=True)
test_df.dropna(axis=1, how='all', inplace=True)

# Add RUL values to data
test_df['RUL'] = test_df['engine_id'].map(lambda x: rul_df.loc[x - 1, 'RUL'])
train_df['RUL'] = train_df['engine_id'].map(lambda x: rul_df.loc[x - 1, 'RUL'])

# train_df, test_df = preprocess_data(train_df, test_df)

In [4]:
# Define features and target variable
X = train_df.drop(columns=['RUL', 'engine_id', 'time_in_cycles'])  # Features (drop non-feature columns) - all columns except the target column
y = train_df['RUL']  # Target (RUL)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
rf_model = RandomForestRegressor(random_state=42)

# Hyperparameters for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print("Best Random Forest Parameters:", grid_search.best_params_)


c:\Users\amurd\miniconda3\envs\pm-env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
108 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
51 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\amurd\miniconda3\envs\pm-env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\amurd\miniconda3\envs\pm-env\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\amurd\miniconda3\envs\pm-env\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraint

Best Random Forest Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [7]:
# Initialize and train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=20, min_samples_leaf=2, min_samples_split=2)
rf_model.fit(X_train, y_train)
val_predictions = rf_model.predict(X_val)
rf_mae = mean_absolute_error(y_val, val_predictions)
rf_rmse = mean_squared_error(y_val, val_predictions, squared=False)
print(f"Random Forest - MAE: {rf_mae}, RMSE: {rf_rmse}")

Random Forest - MAE: 35.61750525752866, RMSE: 41.09045105150293
